In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import math

In [ ]:
import sys
sys.path.append('../src/main')

In [ ]:
from sklearn.model_selection import train_test_split
from dsbase.ModelDSBase import ModelDSBaseWrapper

In [ ]:
from dsbase.models.regression.LightGradientBoostingRegressionDSBase import LightGradientBoostingRegressionDSBaseModel
from dsbase.models.regression.LightGradientBoostingRegressionDSBase import LightGradientBoostingRegressionDSBaseModelParamsToMap

# Data Loading 

In [ ]:
dfA = pd.read_csv('../datasets/predict-sales/dataset1A.csv')
dfB = pd.read_csv('../datasets/predict-sales/dataset1B.csv')
df = pd.concat([dfA,dfB], axis=0)

In [ ]:
df.drop(['Unnamed: 0'], inplace=True, axis=1)

# Data Cleaning 

In [ ]:
def cleanDataset(df):
    # Removing targets without evidence
    df_cleaned = df[~df['date_block_num'].isna()]
    
    # Imputing features
    df_cleaned['STD(sales.item_cnt_day)_imputed'] = df_cleaned['STD(sales.item_cnt_day)'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['STD(sales.item_price)_imputed'] = df_cleaned['STD(sales.item_price)'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['STD(sales.item_cnt_day)_na_indicator'] = df_cleaned['STD(sales.item_cnt_day)'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned['STD(sales.item_price)_na_indicator'] = df_cleaned['STD(sales.item_price)'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned['SKEW(sales.item_cnt_day)_imputed'] = df_cleaned['SKEW(sales.item_cnt_day)'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['SKEW(sales.item_price)_imputed'] = df_cleaned['SKEW(sales.item_cnt_day)'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['SKEW(sales.item_cnt_day)_na_indicator'] = df_cleaned['SKEW(sales.item_cnt_day)'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned['SKEW(sales.item_price)_na_indicator'] = df_cleaned['SKEW(sales.item_cnt_day)'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned['skew_shop_cat_day_imputed'] = df_cleaned['skew_shop_cat_day'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['skew_shop_cat_item_price_imputed'] = df_cleaned['skew_shop_cat_item_price'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned['skew_shop_cat_day_na_indicator'] = df_cleaned['skew_shop_cat_day'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned['skew_shop_cat_item_price_na_indicator'] = df_cleaned['skew_shop_cat_item_price'].apply(lambda x: 1 if (math.isnan(x)) else 0)
    df_cleaned.drop(labels=['STD(sales.item_cnt_day)','STD(sales.item_price)',
                            'SKEW(sales.item_cnt_day)','SKEW(sales.item_price)',
                            'skew_shop_cat_day','skew_shop_cat_item_price'
                           ], inplace=True, axis=1)
    
    # imputing target
    df_cleaned['target_imputed'] = df_cleaned['target'].apply(lambda x: 0 if (math.isnan(x)) else x)
    df_cleaned.drop(labels=['target'], inplace=True, axis=1)
    
    return df_cleaned

In [ ]:
df_cleaned = cleanDataset(df)

# Preparing data for ML 

In [ ]:
X = df_cleaned.drop(labels=['target_imputed'], axis=1).values
y = df_cleaned['target_imputed'].values

# Simple case (Just first test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#params = LightGradientBoostingRegressionDSBaseModelParamsToMap(max_depth=31, n_estimators=100, learning_rate=0.1,num_leaves=31, subsample_for_bin=200000, reg_alpha=0, reg_lambda=0)
params = LightGradientBoostingRegressionDSBaseModelParamsToMap(max_depth=31, n_estimators=100, learning_rate=0.1,num_leaves=31, subsample_for_bin=200000, reg_alpha=3000, reg_lambda=10)
#params = LightGradientBoostingRegressionDSBaseModelParamsToMap(max_depth=20, n_estimators=70, learning_rate=0.1, num_leaves=31, subsample_for_bin=1000000, reg_alpha=100000, reg_lambda=100)
lgbr = ModelDSBaseWrapper('LGBR',X_train,y_train,X_test, y_test,[20,30,40,50,60,70,80,90,100],LightGradientBoostingRegressionDSBaseModel,params)
#lgbr = ModelDSBaseWrapper('LGBR',X_train,y_train,X_test, y_test,[100],LightGradientBoostingRegressionDSBaseModel,params)

In [ ]:
lgbr.train()

In [ ]:
lclgdbr=lgbr.getLearningCurves()

In [ ]:
plt.plot(lclgdbr[0,:],'b',lclgdbr[1,:],'r')

In [ ]:
lgbr.getScore()

# Ensembling

We will not do it finally, for the moment! 

In [ ]:
def processModel(databaseName, modelName):
    dfA = pd.read_csv('../datasets/predict-sales/' + databaseName + 'A.csv')
    dfB = pd.read_csv('../datasets/predict-sales/' + databaseName + 'B.csv')
    df = pd.concat([dfA,dfB], axis=0)
    df.drop(['Unnamed: 0'], inplace=True, axis=1)
    df_cleaned = cleanDataset(df)
    X = df_cleaned.drop(labels=['target_imputed'], axis=1).values
    y = df_cleaned['target_imputed'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    params = LightGradientBoostingRegressionDSBaseModelParamsToMap(max_depth=31, n_estimators=100, learning_rate=0.1,num_leaves=31, subsample_for_bin=200000, reg_alpha=0, reg_lambda=0)
    lgbr = ModelDSBaseWrapper(modelName,X_train,y_train,X_test, y_test,[100],LightGradientBoostingRegressionDSBaseModel,params)
    lgbr.train()
    lclgdbr=lgbr.getLearningCurves()
    print('Score for',modelName,lgbr.getScore())
    return lgbr

## Phase 1 (LightGB for each 1/2 dataset)

In [ ]:
model1 = processModel('dataset1','model1')
model1.save()

In [ ]:
model2 = processModel('dataset2','model2')
model2.save()

In [ ]:
model3 = processModel('dataset3','model3')
model3.save()

In [ ]:
model4 = processModel('dataset4','model4')
model4.save()

In [ ]:
model5 = processModel('dataset5','model5')
model5.save()

In [ ]:
model6 = processModel('dataset6','model6')
model6.save()

## Phase 2 (NN for the 1/2 processed dataset)

Difficult to generate the dataset ... !!!

### Generating 2º dataset processing datasetXB in the previous models

### Training dataset 

# End of Case! 